In [8]:
import osmnx as ox
import networkx as nx
import multiprocessing as mp
import random
import matplotlib.pyplot as plt
import uuid
import json
import os
import time
import ipywidgets as widgets
from IPython.display import display
from sklearn.cluster import KMeans
import numpy as np
from functools import lru_cache
from concurrent.futures import ThreadPoolExecutor
ox.settings.log_console=True
ox.settings.use_cache=True


In [9]:
Graphs = {} # Graphes de chaque region générés par OSMNX
"""
Exemple de Graph: {"Bouches-du-Rhône": <networkx.classes.multidigraph.MultiDiGraph object at 0x000001F4F6F4F4C0>, "Var": <networkx.classes.multidigraph.MultiDiGraph object at 0x000001F4F6F4F4C0>}
"""
Colis = {} # Dictionnaire des colis de chaque region
"""
Exemple de Colis: {"Bouches-du-Rhône": {"U1I2N4V5G1": {"Poids": 1, "Volume": 1,"Position": (43.296174, 5.369953),"PositionNoeud" : 136342614216,"Type": 0 , "RangeLivraison": (1300,1400) }, "U1I2N4V5G2": {"Poids": 1, "Volume": 1,"Position": (43.296174, 5.369953),"Type": 0, "RangeLivraison": "None" }}, "Var": {"U1I2N4V5G1": {"Poids": 1, "Volume": 1,"Position": (43.296174, 5.369953),"Type": 0, "RangeLivraison": (1150,1300) }, "U1I2N4V5G2": {"Poids": 1, "Volume": 1,"Position": (43.296174, 5.369953),"Type": 0, "RangeLivraison": (660,700)}}}
"""
Trajets = {} # Dictionnaire des trajets finaux à effectuer pour chaque region
"""
Exemple de Trajets : {"Bouches-du-Rhône":{ "idtrajet1":{"Colis":(U1I2N4V5G1,U1I2N4V5G2),"Chemin":(2376246743,1948366489,109839872),"Temps": 540 , "Distance": 200}}}
"""
AttributionVehicule = {} # Dictionnaire des attributions de chaque tournée à chaque véhicule
"""
Exemple de AttributionVehicule : {"Bouches-du-Rhône":{ "idtrajet1":"idvehicule1"}}
"""
Vehicules = {} # Dictionnaire des véhicules de chaque region
"""
Exemple de Vehicules : {"idvehicule1":{"PoidsMax": 1000, "Volume": 600,"Type": 0 ,"EmpreinteCarbone": 0.1 }}
"""
Warehouse = {} # Dictionnaire des entrepôts de chaque region
"""
Exemple de Warehouse: {"Bouches-du-Rhône":{ "idwarehouse1":{"Position": (43.296174, 5.369953), "Vehicules" : (idvehicule1,idvehicule2)}}}
"""

"""
Notes :
Les temps sont en minutes depuis minuit
Les distances sont en kilomètres
On s'ajoute comme contrainte que chaque livraison prend du temps à être effectuée
On considère qu'il existe qu'une warehouse par région
Un trajet part d'une warehouse et y revient
@lru_cache(maxsize=None) # Pour mettre en cache les résultats des fonctions
"""

def tempslivraison(volume,poids):
    """
    Fonction qui renvoie le temps de livraison d'un colis en fonction de son volume et de son poids
    """
    if volume < 0.1 and poids < 5 : # Exemple : Une lettre
        return 5
    elif volume < 1 and poids < 10 : # Exemple : Un ordinateur
        return 10
    elif volume < 3 and poids < 100 : # Exemple : Une climatisation
        return 15
    else : # Exemple : Une piscine
        return 30

class TempsGraphRegion :
    """
    Classe permettant de traiter le graphe de la région

    """
    def __init__(self,GraphRegion,ColisRegion,WarehouseRegion,MargeVitesseVehicule):
        """
        Constructeur de la classe TempsGraphRegion
        Calcul tout les temps de trajet entre les points d'intérets du graphe
        Note : On ne veux pas calculer les temps de trajets entre 2 warehouses
        """
        @lru_cache(maxsize=None) # Pour mettre en cache les résultats des fonctions
        def positionclosenoeud(point):
            if point[:2] == "C_":
                return ox.distance.nearest_nodes(self.GraphRegion , ColisRegion[point]["Position"][1],ColisRegion[point]["Position"][0])
            else  :
                return ox.distance.nearest_nodes(self.GraphRegion , WarehouseRegion[point]["Position"][1],WarehouseRegion[point]["Position"][0])

        self.GraphRegion = GraphRegion
        self.TempsGraph = {}
        Point_interet = [[positionclosenoeud(element),element]for element in tuple(WarehouseRegion.keys())+tuple(ColisRegion.keys())]
        ColisNoeud = [[positionclosenoeud(element),element] for element in tuple(ColisRegion.keys())]
        for idpoint1,point1 in Point_interet:
            #On calcul chacun des temps de trajet entre les points d'intérêts
            print("debut calcul :",point1)
            Source = tuple(idpoint1 for i in range(len(ColisNoeud)))
            Target = tuple(ColisNoeud[i][0] for i in range(len(ColisNoeud)))
            TargetID = tuple(ColisNoeud[i][1] for i in range(len(ColisNoeud)))
            chemins = ox.shortest_path(GraphRegion, Source,Target, weight='travel_time') # Thread a mettre ici (, cpu=None)
            TempsTrajets = []
            for chemin in chemins :
                TempsTrajets.append(nx.path_weight(GraphRegion, chemin, weight='travel_time')*(1+MargeVitesseVehicule)/60)
            for temps,cible in zip(TempsTrajets,TargetID) :
                # On ajoute le temps de trajet dans le dictionnaire
                self.TempsGraph[(point1,cible)] = temps
                self.TempsGraph[(cible,point1)] = temps
            #On retire le noeud de la liste des noeuds à traiter
            ColisNoeud = [element for element in ColisNoeud if element[0] != idpoint1]
            print("fin calcul :",point1)
            
        
        print("fin calcul")
        print("TempsGraph : ",self.TempsGraph)

    def graph(self) :
        return self.GraphRegion

    def way(self,origine,destination) :
        """
        Fonction qui renvoie le temps le plus court entre deux points d'intérêt
        origine et destination sont des 
        id de point d'intérêt
        """
        return self.TempsGraph[(origine,destination)]


In [10]:
class TempsGraphRegionTest:
    def __init__(self, GraphRegion, ColisRegion, WarehouseRegion, MargeVitesseVehicule):
        def calculate_shortest_paths(source, target, target_id):
            chemins = ox.shortest_path(GraphRegion, source, target, weight='travel_time')
            TempsTrajets = []
            for chemin in chemins:
                TempsTrajets.append(nx.path_weight(GraphRegion, chemin, weight='travel_time') * (1 + MargeVitesseVehicule) / 60)
            for temps, cible in zip(TempsTrajets, target_id):
                self.TempsGraph[(point1, cible)] = temps
                self.TempsGraph[(cible, point1)] = temps

        @lru_cache(maxsize=None)
        def positionclosenoeud(point):
            if point[:2] == "C_":
                return ox.distance.nearest_nodes(GraphRegion, ColisRegion[point]["Position"][1], ColisRegion[point]["Position"][0])
            else:
                return ox.distance.nearest_nodes(GraphRegion, WarehouseRegion[point]["Position"][1], WarehouseRegion[point]["Position"][0])

        self.GraphRegion = GraphRegion
        self.TempsGraph = {}
        Point_interet = [[positionclosenoeud(element), element] for element in tuple(WarehouseRegion.keys()) + tuple(ColisRegion.keys())]
        ColisNoeud = [[positionclosenoeud(element), element] for element in tuple(ColisRegion.keys())]

        with ThreadPoolExecutor() as executor:
            futures = []
            for idpoint1, point1 in Point_interet:
                source = tuple(idpoint1 for _ in range(len(ColisNoeud)))
                target = tuple(ColisNoeud[i][0] for i in range(len(ColisNoeud)))
                target_id = tuple(ColisNoeud[i][1] for i in range(len(ColisNoeud)))
                future = executor.submit(calculate_shortest_paths, source, target, target_id)
                futures.append(future)

            # Wait for all tasks to complete
            for future in futures:
                future.result()

        print("fin calcul")
        print("TempsGraph: ", self.TempsGraph)

    def graph(self):
        return self.GraphRegion

    def way(self, origine, destination):
        return self.TempsGraph[(origine, destination)]

Hyper Parametrage

In [12]:
from geopy import distance
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import openrouteservice

# Set up geocoders
geolocator = Nominatim(user_agent="my_app")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
you = ("43.623472, 4.981389")

me = ("43.4144504, 5.43006666")
# Convert the coordinates to addresses
you_address = geocode(you)
me_address = geocode(me)

# Set up the routing service
client = openrouteservice.Client('5b3ce3597851110001cf62480d61db042f164490ba52065701c63911')  # Replace with your API key

# Get the route between the two addresses
route = client.directions(
    coordinates=[(you_address.longitude, you_address.latitude),
                 (me_address.longitude, me_address.latitude)],
    profile='driving-car'  # Use 'driving-car' for vehicles
)

# Extract the distance from the route
distance_in_meters = route['routes'][0]['summary']['distance']

# Convert distance to miles
distance_in_miles = distance.distance(meters=distance_in_meters).miles

print(f"Miles: {distance_in_miles:.2f}")


Miles: 39.92


Lexture des Hyperparamètres via le fichier settings.txt

In [14]:
#Path_settings = "C:/Users/alexa/Documents/GitHub/TSP/Projet/settings.txt"
#lecture_hyperparametre(Path_settings)

Phase 1/2 : Saving/Load Graphs

In [15]:
def save_graphs(departments,force_download=False):
    # Obtenir le répertoire de travail actuel
    current_dir = os.getcwd()
    
    # Chemin complet du répertoire "graphs"
    destination_path = os.path.join(current_dir, 'graphs')
    
    # Vérifier si le répertoire "graphs" existe, sinon le créer
    if not os.path.exists(destination_path):
        os.makedirs(destination_path)
    
    for department in departments:
        
        # Créer un nom de fichier basé sur le département
        file_name = f'graph_{department}.graphml'
        
        # Définir le chemin de destination complet pour sauvegarder le graphe
        file_path = os.path.join(destination_path, file_name)
        
        #On vérifie si le fichier existe déjà
        if os.path.exists(file_path) and not force_download:
            print(f"Le graphe du département {department} existe déjà à l'emplacement : {file_path}")
        else :
            print(f"Téléchargement du graphe du département {department} en cours...")
            # Télécharger le graphe du département spécifié
            graph = ox.graph_from_place(f'{department}, France', network_type='drive')
            # Sauvegarder le graphe au format GraphML
            ox.save_graphml(graph, filepath=file_path)
        print(f"Graphe du département {department} sauvegardé à l'emplacement : {file_path}")


def load_graphs(departments):
    Graphs = {}
    # Obtenir le répertoire de travail actuel
    current_dir = os.getcwd()
    
    # Chemin complet du répertoire "graphs"
    source_path = os.path.join(current_dir, 'graphs')
    
    for department in departments:
        # Créer un nom de fichier basé sur le département
        file_name = f'graph_{department}.graphml'
        
        # Définir le chemin complet du fichier à charger
        file_path = os.path.join(source_path, file_name)
        
        if os.path.exists(file_path):
            # Charger le graphe depuis le fichier
            Graphs[department] = ox.load_graphml(filepath=file_path)
            #   Ajouter les vitesses aux arêtes
            Graphs[department] = ox.add_edge_speeds(Graphs[department]) # TODO : A etudier
            # Ajouter les temps de trajet aux arêtes
            Graphs[department] = ox.add_edge_travel_times(Graphs[department]) # TODO : A etudier
            print(f"Graphe du département {department} chargé depuis : {file_path}")
        else:
            print(f"Aucun fichier de graphe trouvé pour le département {department}")
    return Graphs

Load :

/!\ Prend beaucoup de RAM si plusieurs départements sont chargés.

In [16]:
Graphs = load_graphs([REGION])

Aucun fichier de graphe trouvé pour le département Bouches-du-Rhône


Sauvegarder tout les departements :

In [17]:
print(Departements)
#save_graphs(Departements,False)

['Ain', 'Aisne', 'Allier', 'Alpes-de-Haute-Provence', 'Hautes-Alpes', 'Alpes-Maritimes', 'Ardèche', 'Ardennes', 'Ariège', 'Aube', 'Aude', 'Aveyron', 'Bouches-du-Rhône', 'Calvados', 'Cantal', 'Charente', 'Charente-Maritime', 'Cher', 'Corrèze', 'Corse-du-Sud', 'Haute-Corse', "Côte-d'Or", "Côtes-d'Armor", 'Creuse', 'Dordogne', 'Doubs', 'Drôme', 'Eure', 'Eure-et-Loir', 'Finistère', 'Gard', 'Haute-Garonne', 'Gers', 'Gironde', 'Hérault', 'Ille-et-Vilaine', 'Indre', 'Indre-et-Loire', 'Isère', 'Jura', 'Landes', 'Loir-et-Cher', 'Loire', 'Haute-Loire', 'Loire-Atlantique', 'Loiret', 'Lot', 'Lot-et-Garonne', 'Lozère', 'Maine-et-Loire', 'Manche', 'Marne', 'Haute-Marne', 'Mayenne', 'Meurthe-et-Moselle', 'Meuse', 'Morbihan', 'Moselle', 'Nièvre', 'Nord', 'Oise', 'Orne', 'Pas-de-Calais', 'Puy-de-Dôme', 'Pyrénées-Atlantiques', 'Hautes-Pyrénées', 'Pyrénées-Orientales', 'Bas-Rhin', 'Haut-Rhin', 'Rhône', 'Haute-Saône', 'Saône-et-Loire', 'Sarthe', 'Savoie', 'Haute-Savoie', 'Paris', 'Seine-Maritime', 'Seine-

Génération

In [18]:
#Définition des fonctions
def get_random_coordinates_on_edge(graph, edges):
    random_edge = random.choice(edges)
    u, v = random_edge[0], random_edge[1]
    u_lat, u_lon = graph.nodes[u]['y'], graph.nodes[u]['x']
    v_lat, v_lon = graph.nodes[v]['y'], graph.nodes[v]['x']
    lat = random.uniform(u_lat, v_lat)
    lon = random.uniform(u_lon, v_lon)
    return lat, lon


Génération Colis

In [19]:
def generate_random_points(graph, n_points):
    # Récupération des arêtes du graphe
    edges = list(graph.edges())

    # Configuration des valeurs de poids et de volume pour chaque dimension de point
    point_configurations = {
        'petit': {'weight': COLIS_PETIT_POIDS, 'volume': COLIS_PETIT_VOLUME},
        'moyen': {'weight': COLIS_MOYEN_POIDS, 'volume': COLIS_MOYEN_VOLUME},
        'grand': {'weight': COLIS_GRAND_POIDS, 'volume': COLIS_GRAND_VOLUME}
    }

    # Mapping des types de point vers des valeurs numériques
    type_mapping = {'normal': 0, 'réfrigéré': 1, 'exceptionnel': 2}

    # Initialisation du dictionnaire des données des colis
    Colis = {REGION: {}}

    # Génération des points aléatoires
    for _ in range(n_points):
        # Sélection aléatoire de la dimension du point (petit, moyen, grand)
        point_dimension = random.choices(['petit', 'moyen', 'grand'], weights=PROBA_TAILLE, k=1)[0]

        # Sélection aléatoire du type de point (normal, réfrigéré, exceptionnel)
        point_type = random.choices(['normal', 'réfrigéré', 'exceptionnel'], weights=PROBA_TYPE, k=1)[0]

        # Mapping du type de point vers une valeur numérique
        point_type_num = type_mapping[point_type]

        # Récupération de la plage de poids et de volume pour la dimension du point
        weight_range = point_configurations[point_dimension]['weight']
        volume_range = point_configurations[point_dimension]['volume']

        # Vérification du type de point pour attribuer des valeurs "exceptionnelles" au poids et au volume
        if point_type == 'exceptionnel':
            weight_range = COLIS_EXCEPT_POIDS
            volume_range = COLIS_EXCEPT_VOLUME

        # Génération aléatoire du poids et du volume dans les plages définies
        weight = round(random.uniform(*weight_range), 2)
        volume = round(random.uniform(*volume_range), 2)

        # Génération aléatoire des coordonnées du point sur une arête du graphe
        lat, lon = get_random_coordinates_on_edge(graph, edges)
        

        # Génération aléatoire de la plage de livraison (temps minimum et maximum)
        if random.random() < PROBA_NO_RANGE:
            range_livraison = (RANGE_MIN_TIME, RANGE_MAX_TIME)  # Pas de plage de livraison
        else:
            min_time = random.randint(RANGE_MIN_TIME, RANGE_MAX_TIME - RANGE_MARGE)
            max_time = random.randint(min_time + RANGE_MARGE, RANGE_MAX_TIME)
            range_livraison = (min_time, max_time)

        # Génération d'un identifiant unique pour le colis
        colis_id = "C_" + str(uuid.uuid4())[:10].replace("-", "").upper()

        # Construction des informations du colis dans un dictionnaire
        colis_info = {
            "Poids": weight,
            "Volume": volume,
            "Position": (lat, lon),
            "Type": point_type_num,
            "RangeLivraison": range_livraison
        }

        # Ajout des informations du colis dans le dictionnaire des données des colis
        Colis[REGION][colis_id] = colis_info



    # Retourne le dictionnaire des données des colis générées
    return Colis



Génération Véhicule

In [20]:
def CreationVehicules(nombre_total_vehicules, repartition_vehicules):
    vehicules_crees = 0
    vehicules = {}
    list_veh_uuid = []
    # Calculer le nombre de véhicules à créer pour chaque type en fonction du taux de répartition
    for vehicule, taux in repartition_vehicules.items():
        nombre_vehicules = int(nombre_total_vehicules * taux)
        for _ in range(nombre_vehicules):
            vehicule_uuid = f"V_{str(uuid.uuid4())}"
            vehicules[vehicule_uuid] = {
                "PoidsMax": Vehicules[vehicule]["PoidsMax"],
                "Volume": Vehicules[vehicule]["Volume"],
                "Type": vehicule,
                "EmpreinteCarbone": Vehicules[vehicule]["EmpreinteCarbone"]
            }
            list_veh_uuid.append(vehicule_uuid)
            vehicules_crees += 1

    # Compléter le nombre de véhicules si nécessaire pour atteindre le nombre total défini
    while vehicules_crees < nombre_total_vehicules:
        vehicule = random.choice(list(repartition_vehicules.keys()))
        vehicule_uuid = str(uuid.uuid4())
        vehicules[vehicule_uuid] = {
            "PoidsMax": Vehicules[vehicule]["PoidsMax"],
            "Volume": Vehicules[vehicule]["Volume"],
            "Type": vehicule,
            "EmpreinteCarbone": Vehicules[vehicule]["EmpreinteCarbone"]
        }
        list_veh_uuid.append(vehicule_uuid)
        vehicules_crees += 1

    # Afficher les véhicules créés
    #for vehicule_uuid, vehicule_data in vehicules.items():
        #print(f"UUID : {vehicule_uuid} | Véhicule : {vehicule_data['Type']}")
    
    return vehicules, list_veh_uuid
VehiculesList = CreationVehicules(nombre_total_vehicules, repartition_vehicules)


Génération Entrepots


In [21]:
def generate_entrepots(graph, donnees_colis, nombre_entrepots):
    # Récupération des données de la région spécifiée dans le dictionnaire des données des colis
    donnees_region = donnees_colis.get(REGION, {})
    coords_colis = []

    # Parcours des colis dans la région spécifiée
    for colis_id, infos_colis in donnees_region.items():
        position = infos_colis['Position']
        lat, lon = position
        coords_colis.append([lat, lon])

    # Application de l'algorithme K-means pour trouver les centres des clusters
    kmeans = KMeans(n_clusters=nombre_entrepots, n_init=10)
    kmeans.fit(coords_colis)
    centres = kmeans.cluster_centers_

    # Initialisation du dictionnaire des données des entrepôts
    Warehouse = {REGION: {}}
    for centre in centres:
        lat, lon = centre

        # Trouver le noeud le plus proche du centre du cluster sur le réseau routier
        nearest_node = ox.distance.nearest_nodes(graph, lon, lat)
        nearest_coords = graph.nodes[nearest_node]['y'], graph.nodes[nearest_node]['x']

        # Ajuster les coordonnées de l'entrepôt pour qu'il se trouve sur un point de route
        lat, lon = nearest_coords
        
        id_entrepot = "WH_" + str(uuid.uuid4())[:10].replace("-", "").upper()
        vehicules, list_veh_uuid = CreationVehicules(nombre_total_vehicules, repartition_vehicules)
        infos_entrepot = {
            "Position": (lat, lon),
            "Vehicules": list_veh_uuid
        }
        Warehouse[REGION][id_entrepot] = infos_entrepot

    return Warehouse


In [22]:
def trace_points_et_entrepots_sur_graphe(graph, donnees_colis, donnees_entrepots):
    # Création de la figure et de l'axe pour le graphe
    fig, ax = ox.plot_graph(graph, figsize=(10, 10), show=False, close=False, bgcolor='white', edge_color='black', edge_linewidth=0.5, node_size=0, edge_alpha=0.2)

    # Récupération des données de la région spécifiée dans le dictionnaire des données des colis
    donnees_region = donnees_colis.get(REGION, {})
    couleurs_types = {0: 'red', 1: 'blue', 2: 'yellow'}

    # Parcours des colis dans la région spécifiée
    for colis_id, infos_colis in donnees_region.items():
        position = infos_colis['Position']
        type_colis = infos_colis['Type']
        couleur = couleurs_types.get(type_colis, 'black')
        lat, lon = position
        ax.scatter(lon, lat, c=couleur, s=25)
        


    # Parcours des entrepôts dans la région spécifiée
    entrepots_region = donnees_entrepots.get(REGION, {})
    for entrepot_id, infos_entrepot in entrepots_region.items():
        position = infos_entrepot['Position']
        lat, lon = position
        ax.scatter(lon, lat, c='lime', s=100)
        
        

    # Ajout des légendes
    ax.scatter([], [], c='red', s=25, label='Colis normaux')
    ax.scatter([], [], c='blue', s=25, label='Colis réfrigérés')
    ax.scatter([], [], c='yellow', s=25, label='Colis exceptionnels')
    ax.scatter([], [], c='lime', s=100, label='Entrepôts')

    # Affichage de la légende et du graphe
    ax.legend()
    plt.show()


In [23]:
Colis = generate_random_points(Graphs[REGION], N_POINTS)
print(Colis)

KeyError: 'Bouches-du-Rhône'

In [ ]:
print(Colis)

In [ ]:
Warehouse = generate_entrepots(Graphs[REGION], Colis, NOMBRE_ENTREPOTS)

In [ ]:
print(Warehouse)

Si besoin d'afficher

In [ ]:
trace_points_et_entrepots_sur_graphe(Graphs[REGION], Colis, Warehouse)

Phase 3 : Calculs des chemins via ANTS (Alexandre)

In [ ]:
class PheromoneRegion :
    """
    Classe permettant de gérer les phéromones sur le graphe de la région
    Pheromone est sous la forme : (idpoint1,idpoint2) : pheromone
    """
    def __init__(self,ColisRegion,WarehouseRegion):
        #Une matrice de phéromone par région
        self.Pheromone = {}
        # On initialise les phéromones
        for id1 in list(ColisRegion.keys()) + (list(WarehouseRegion.keys())):
            for id2 in list(ColisRegion.keys()) + (list(WarehouseRegion.keys())):
                if id1[:3] == "WH_" and id2[:3] == "WH_" : # On veux pas de phéromone entre deux entrepots
                    continue
                self.Pheromone[(id1,id2)] = 0 if id1 == id2 else 1
        #print("test init pheromone :",self.Pheromone)

    def scores(self,TempsGraphRegion,Colis_Possible,Id_Colis, alpha=ALPHA,beta=BETA) :
        """
        Renvoie les scores de chaque colis possible pour la fourmi
        """
        Scores = []
        for idcolis in Colis_Possible :
            Scores += ((self.Pheromone[(Id_Colis,idcolis)])**alpha) * ((1/TempsGraphRegion[Id_Colis][idcolis])**beta)
        return Scores

    def update(self,TrajetsComplet,Distance,gamma=GAMMA):
        # On met à jour les phéromones à partir des id des point d'interet
        for trajet in TrajetsComplet :
            identrepot = TrajetsComplet[0][0]
            listeinteret = [identrepot] + trajet[2]
            for i in range(len(listeinteret)-1) :
                self.Pheromone[(listeinteret[i],listeinteret[i+1])] += gamma * (1/Distance)



class Seen :
    """
    Garde en mémoire si la fourmi est passer par un colis
    """
    def __init__(self,ColisRegion) :
        self.Seen = {}
        for idcolis in ColisRegion.keys() :
            self.Seen[idcolis] = False
    def update(self,idnoeud) :
        self.Seen[idnoeud] = True
    def isseen(self,idnoeud) :
        return self.Seen[idnoeud]

def deepcopy(dict) :
    return json.loads(json.dumps(dict))

def calcscoretournee(Trajets):
    return sum(trajet[4] for trajet in Trajets)

def calchemin(GraphRegion,Id_Point1,Id_Point2) :
    return ox.shortest_path(GraphRegion,Id_Point1,Id_Point2,weight="travel_time")
"""
ALGORITHME DES FOURMIS MODIFIE
"""
class algofourmis :
    """
    On cherche à minimiser le temps de trajet ainsi,
    on cherche donc à minimiser le temps de trajet total.
    """
    def create(self, Colis, Warehouse, StartTime=RANGE_MIN_TIME, EndTime=RANGE_MAX_TIME):
        TrajetsComplet = {}
        # Création des trajets pour chaque région
        for nom_region in Graphs.keys() :
            print("On traite la region :",nom_region)
            # On effectue l'algorithme pour chaque région
            G_region = Graphs[nom_region]
            # On récupère les colis de la région
            colis_region = Colis[nom_region]
            # On récupère les entrepôts de la région
            warehouse_region = Warehouse[nom_region]
            # On gènère l'objet qui gère les temps de trajet
            GraphRegion = TempsGraphRegion(G_region,colis_region,warehouse_region,MARGEVITESSEVEHICULE)
            # On lance l'algorithme des fourmis
            TrajetsComplet[nom_region] = fourmi_region(colis_region,warehouse_region,GraphRegion,StartTime,EndTime)
        
        # On formate les trajets
        TrajetsFormat = {}
        """
        Exemple de Trajets : {"Bouches-du-Rhône":{ "idtrajet1":{"Colis":(U1I2N4V5G1,U1I2N4V5G2),"Chemin":(2376246743,1948366489,109839872),"Temps": 540,"Type" : 0 , "Distance": 200}}}
        Exemple de tournee : [[identrepot1,type,[idcolis1,...],score1],...]
        """
        for nom_region in Graphs.keys() :
            TrajetsFormat[nom_region] = {}
            for trajet in TrajetsComplet[nom_region] :
                idtrajet = id()
                TrajetsFormat[nom_region][idtrajet] = {
                    "Colis" : tuple(trajet[2]),
                    "Chemin" : calchemin(Graphs[nom_region],tuple(trajet[2])),
                    "Temps" : trajet[3],
                    "Type" : trajet[1],
                }
        return TrajetsFormat
            

"""
ALGORITHME DES FOURMIS ECHELLE REGION
"""
def fourmi_region(ColisRegion,WarehousesRegion,GraphRegion,StartTime=RANGE_MIN_TIME,EndTime=RANGE_MAX_TIME,Iteration=NOMBRE_ITERATION) :
    # On cherche le meilleur chemin via l'algorithme des fourmis modifié
    # On créer les phéromones
    Pheromones = PheromoneRegion(ColisRegion,WarehousesRegion)
    Bestcalcscore = 0
    for numero_iteration in range(Iteration) :
        # Etape 0 : On initialise les variables
        Clone_Colis = deepcopy(ColisRegion) # On clone les colis pour ne pas les modifier
        Tournee_Candidat = [] # Contient le trajet complet candidat sous la forme [[identrepot1,type,[idcolis1,...],score1],...]

        """
        Etape 1 : On va faire balader la fourmi
        Note : CandidatColis contient les ids des colis à livrer dans l'ordre
        """
        while len(Clone_Colis) > 0 : # Tant qu'il reste des colis à livrer
            CandidatWarehouse,CandidatType,CandidatColis,CandidatScore = fourmi_trajetcandidat(GraphRegion,Pheromones,Clone_Colis,WarehousesRegion) # On récupère les colis et le trajet candidat
            Tournee_Candidat.append([CandidatWarehouse,CandidatType,CandidatColis,CandidatScore]) # On ajoute le trajet candidat
            # On retire les colis livrés au clone
            for idcolis in CandidatColis :
                del Clone_Colis[idcolis]
        """
        Etape 2 : On va actualiser la matrice de phéromones à partir de la liste des trajets candidats
        """
        Pheromones.update(Tournee_Candidat)
        """
        Etape 3 : On conserve la meilleur tournée jusqu'à présent
        """
        ScoreTournee = calcscoretournee(Tournee_Candidat)
        if ScoreTournee < Bestcalcscore :
            Trajet_Best = Tournee_Candidat
            Bestcalcscore = ScoreTournee
            print("Meilleur score actuel :",Bestcalcscore, "à l'itération :",numero_iteration)
            

    return Trajet_Best # Retourne les trajets à effectuer pour la région


"""
ALGORITHME DES FOURMIS CANDIDAT
"""
def fourmi_trajetcandidat(GraphRegion,Pheromone,Colis,WarehouseRegion,StartTime=RANGE_MIN_TIME,EndTime=RANGE_MAX_TIME,MARGETEMPSRETOUR=MARGETEMPSRETOUR):
    CandidatColis = [] #Liste des id des colis pris dans l'ordre de prise
    #On va faire balader la fourmi
    # On choisi un entrepôt au hasard
    identrepot_choisi = random.choice(list(WarehouseRegion.keys()))
    print("ID de l'entrepot choisi :",identrepot_choisi)
    # On choisi un type de colis au hasard
    typecolis_choisi = Colis[list(Colis.keys())[0]]['Type'] # On prend le type du premier colis dans notre dictionnaire
    print("Type de colis choisi :",typecolis_choisi)
    # On initialise la gestion des colis vus
    SeenRegion = Seen(Colis)
    # On filtre les colis en fonction du type de colis choisi
    Colis_Possible = { idcolis : Colis[idcolis] for idcolis in Colis.keys() if Colis[idcolis]['Type'] == typecolis_choisi} 
    # On initialise la position/temps de la fourmi (quel id colis elle est en premier)
    Position_fourmi = identrepot_choisi
    Temps_fourmi = StartTime

    # On fait balader la fourmi jusqu'à ce qu'elle n'ai plus le temps.
    while True:
        print("Nombre de Colis disponibles :",len(Colis_Possible))
        print("Temps de la fourmi :",Temps_fourmi)
        # On filtre les colis déja livrés
        Colis_Possible = { idcolis : Colis_Possible[idcolis] for idcolis in Colis_Possible.keys() if SeenRegion.isseen(idcolis) == False}
        # On filtre les colis (donc livraison) qui ne sont plus disponibles car plus dans la range haute de livraison
        Colis_Possible = {
            idcolis: Colis_Possible[idcolis]
            for idcolis in Colis_Possible
            if Colis_Possible[idcolis]['RangeLivraison'][1]
            <= Temps_fourmi + GraphRegion.way(idcolis, identrepot_choisi)
        }
        # Il n'y a plus de colis disponibles
        if not Colis_Possible:
            break
        # On choisit un colis au hasard en fonction de la distance et des phénomones
        idcolis_choisi = random.choices(list(Colis_Possible.keys()), weights=Pheromone.scores(GraphRegion,Colis_Possible,Position_fourmi), k=1)[0]
        print("ID du colis choisi :",idcolis_choisi)
        # On calcule le temps de trajet + livraison
        Temps_livraison = GraphRegion.way(Position_fourmi,idcolis_choisi) + tempslivraison(Colis_Possible[idcolis_choisi]['Poids'],typecolis_choisi)
        #Escape Case (car boucle infinie)
        if Temps_fourmi+Temps_livraison >= EndTime-MARGETEMPSRETOUR:
            # La fourmi n'a plus le temps de retourner à l'entrepôt, on accepte donc pas la dernière livraison
            break
        # La fourmi a encore le temps de retourner à l'entrepôt, on accepte donc la livraison choisie
        # On ajoute le colis à la liste des colis à livrer
        CandidatColis.append(idcolis_choisi)
        # On ajoute le temps de trajet + livraison à la fourmi
        Temps_fourmi += Temps_livraison
        # On change de position la fourmi
        Position_fourmi = idcolis_choisi
        # On ajoute le colis à au registre des colis livrés
        SeenRegion.update(idcolis_choisi)
        print("Temps actuel de la fourmi :",Temps_fourmi)
    # On retourne le trajet candidat
    return identrepot_choisi,typecolis_choisi,CandidatColis,Temps_fourmi-StartTime

Tests :

In [ ]:
algofourmis.create(Graphs,Colis,Warehouse)

In [ ]:
# (1570926434, 8212629471): 4089.1199999999976, (1570926434, 739507540): 3650.52
# On plot le plus cours chemin en travel_time entre 2 noeuds

Phase 4 : Attribution des camions (Clement)

In [ ]:

Trajetexemple = ({"Bouches-du-Rhône":
                      { "idtrajet1":
                                          {"Colis":("C_1F3C8F827","C_E112827A6"),
                                           "Chemin":(2376246743,1948366489,109839872),
                                           "Temps": 250 , 
                                           "Distance": 50},
                        "idtrajet2":
                                        {"Colis":("C_1F3C8F827","C_E112827A6"),
                                           "Chemin":(2376246743,1948366489,109839872),
                                           "Temps": 500 , 
                                           "Distance": 100},
                        "idtrajet3":
                                        {"Colis":("C_EDD10B9D4","C_EDD10B9D4"),
                                           "Chemin":(2376246743,1948366489,109839872),
                                           "Temps": 750 , 
                                           "Distance": 200}},       
                    })
Colis = {'Bouches-du-Rhône': 
         {'C_1F3C8F827': 
          {'Poids': 0.05, 
           'Volume': 0.02, 
           'Position': (43.87242775847219, 4.860431950821716), 
           'PositionNoeud': 1832539677, 
           'Type': 0, 
           'RangeLivraison': (480, 1080)}, 
          'C_E112827A6': 
           {'Poids': 7.68, 
            'Volume': 0.03, 
            'Position': (43.3691969566393, 5.24911716336601), 
            'PositionNoeud': 2322856070, 
            'Type': 0, 
            'RangeLivraison': (945, 1016)}, 
           'C_EDD10B9D4': 
            {'Poids': 0.69, 
             'Volume': 0.01, 
             'Position': (43.24822166385922, 5.59845632058536), 
             'PositionNoeud': 252931723, 
             'Type': 1, 
             'RangeLivraison': (480, 1080)}}}
"0 Normaux, 1 réfrigérés, 2 exceptionnels"

#Temps en min, dist en km
Warehouse = {'Bouches-du-Rhône': 
             {'WH_2815E24DC': 
              {'Position': (43.393074596100604, 5.4857390627666875), 
               'PositionNoeud': 278061257, 
               'Vehicules': ['V_c5c72ba2-d4fe-4a98-bef3-2a4fbe1d0ff5', 
                              'V_c3b73e39-24b9-4beb-a03d-8e4adaa60b96', 
                              'V_d6903311-7c19-48e7-ad07-ba92e190b2c5', 
                              'V_043519ef-1c14-4870-a1c9-02ce114ac53d', 
                              'V_f1f13307-719f-40eb-bbfd-acde53dd22de']}, 
                'WH_22BC3D2C3': 
                {'Position': (43.50198905220718, 5.077744505907566), 
                 'PositionNoeud': 278061257, 
                 'Vehicules': ['V_c5c72ba2-d4fe-4a98-bef3-2a4fbe1d0ff5', 
                               'V_c3b73e39-24b9-4beb-a03d-8e4adaa60b96', 
                               'V_d6903311-7c19-48e7-ad07-ba92e190b2c5', 
                               'V_043519ef-1c14-4870-a1c9-02ce114ac53d', 
                               'V_f1f13307-719f-40eb-bbfd-acde53dd22de']}}}
VehiculesList = ({'V_c5c72ba2-d4fe-4a98-bef3-2a4fbe1d0ff5': 
                  {'PoidsMax': 0.05, 
                   'Volume': 0.02, 
                   'Type': 0, 
                   'EmpreinteCarbone': 0.25}, 
                'V_c3b73e39-24b9-4beb-a03d-8e4adaa60b96': 
                {'PoidsMax': 3500, 
                 'Volume': 20, 
                 'Type': 0, 
                 'EmpreinteCarbone': 0.6}, 
                'V_d6903311-7c19-48e7-ad07-ba92e190b2c5': 
                {'PoidsMax': 7500, 
                 'Volume': 40, 
                 'Type': 0, 
                 'EmpreinteCarbone': 0.9}, 
                 'V_043519ef-1c14-4870-a1c9-02ce114ac53d': 
                {'PoidsMax': 1200, 
                 'Volume': 5, 
                 'Type': 1, 
                 'EmpreinteCarbone': 0.4}, 
                 'V_f1f13307-719f-40eb-bbfd-acde53dd22de': 
                {'PoidsMax': 3500, 
                 'Volume': 20, 
                 'Type': 1, 
                 'EmpreinteCarbone': 0.75}
                })

def choisir_vehicules_pour_trajets(trajets, colis, entrepots, vehicules):
    meilleurs_choix_vehicules = {}
    vehicules_assignes = set()
    vehicules_restants = {}
    for departement, trajet in trajets.items():
        meilleurs_choix_vehicules[departement] = {} #Initialisation
        vehicules_restants[departement] = {} #Initialisation

        for id_trajet, info_trajet in trajet.items():
            colis_trajet = info_trajet["Colis"]
            poids_total, volume_total, type_colis, empreinte_carbone_min, meilleur_vehicule\
            = obtenir_meilleur_vehicule(colis_trajet, colis, departement, entrepots, vehicules, vehicules_assignes)

            if meilleur_vehicule is not None:
                vehicules_assignes.add(meilleur_vehicule)
                meilleurs_choix_vehicules[departement][id_trajet] = meilleur_vehicule
                poids_restant, volume_restant = calculer_poids_volume_restant(vehicules, meilleur_vehicule, poids_total, volume_total)
                vehicules_restants = mettre_a_jour_vehicules_restants(vehicules_restants, departement, meilleur_vehicule, poids_restant, volume_restant)

    return meilleurs_choix_vehicules, vehicules_restants

def obtenir_meilleur_vehicule(colis_trajet, colis, departement, entrepots, vehicules, vehicules_assignes):
    poids_total = 0
    volume_total = 0
    type_colis = None
    empreinte_carbone_min = float('inf')
    meilleur_vehicule = None

    for colis_id in colis_trajet: #Recuperation informations colis dans un trajet
        colis_info = colis[departement][colis_id]
        poids = colis_info["Poids"]
        volume = colis_info["Volume"]
        colis_type = colis_info["Type"]
        poids_total += poids #Poids total des colis dans le trajet
        volume_total += volume #Volume total des colis dans le trajet

        if type_colis is None:
            type_colis = colis_type
        elif type_colis != colis_type:
            type_colis = -1 #Colis non valide

    for entrepot, info_entrepot in entrepots[departement].items(): 
        vehicules_entrepot = info_entrepot["Vehicules"]

        for vehicule_id in vehicules_entrepot:
            if vehicule_id in vehicules_assignes: #Verification de ne pas assigner un véhicule 2 fois
                continue
            #Récupération des infos de chaque véhicule
            vehicule_info = vehicules[vehicule_id]
            vehicule_poids_max = vehicule_info["PoidsMax"]
            vehicule_volume_max = vehicule_info["Volume"]
            vehicule_type = vehicule_info["Type"]
            #Verification si véhicule capable d'avoir tout les colis du trajet
            if vehicule_type == type_colis and vehicule_poids_max >= \
                poids_total and vehicule_volume_max >= volume_total:

                vehicule_empreinte_carbone = vehicule_info["EmpreinteCarbone"]
                #Verification meilleure empreinte carbone véhicule
                if vehicule_empreinte_carbone < empreinte_carbone_min:
                    empreinte_carbone_min = vehicule_empreinte_carbone
                    meilleur_vehicule = vehicule_id

    return poids_total, volume_total, type_colis, empreinte_carbone_min, meilleur_vehicule

def calculer_poids_volume_restant(vehicules, vehicule_id, poids_max, volume_max):
    vehicule_poids_max = vehicules[vehicule_id]["PoidsMax"]
    vehicule_volume_max = vehicules[vehicule_id]["Volume"]
    poids_restant = vehicule_poids_max - poids_max
    volume_restant = vehicule_volume_max - volume_max
    return poids_restant, volume_restant

def mettre_a_jour_vehicules_restants(vehicules_restants, departement, vehicule_id, poids_restant, volume_restant):
    vehicules_restants.setdefault(departement, {}).setdefault(vehicule_id, {}).update({
        "Poids restant": poids_restant,
        "Volume restant": volume_restant
    })

    return vehicules_restants

def trouver_colis_restants(colis_trajet, colis_departement):
    for colis_id in colis_trajet:
        if colis_id not in colis_departement:
            return colis_id

resultat, vehicules_restants = choisir_vehicules_pour_trajets(Trajetexemple, Colis, Warehouse, VehiculesList)

print("Meilleurs véhicules par trajet:")
print(resultat)
print()
print("Poids et volume restant dans les véhicules:")
print(vehicules_restants)